dfdfasdf

# AI Function

Databricks에 저장된 데이터에 AI를 적용 할 수 있는 내장 함수이며, Databricks SQL, Notebook, Lakeflow, workflow 등 어디서든 실행 가능

General-Purpose function과 Task Specific functions 크게 2 유형으로 분류

General-Purpose Function
- ai_query

Task-Specific Function
- ai_analyze_sentiment	텍스트 감정 분석
- ai_classify	텍스트를 제공한 라벨에 따라 분류
- ai_extract	지정된 엔티티 추출
- ai_fix_grammar	문법 오류 수정
- ai_gen	사용자 프롬프트에 대한 응답 생성
- ai_mask	지정된 엔티티 마스킹
- ai_parse_document (Beta)	비정형 문서에서 구조적 콘텐츠 추출
- ai_similarity	두 문자열의 의미적 유사도 계산
- ai_summarize	텍스트 요약
- ai_translate	텍스트 번역
- ai_forecast	시계열 데이터 예측
- vector_search	Mosaic AI 벡터 검색 인덱스 조회


## 일반 용도의 AI Function

- UDF에서 활용 가능
- LDP에서 활용 가능하나 Table 속성 추가 필요
- 전통적인 ML Model도 호출 가능
- Custom Models과 External Models 사용은 Preview 기능
- https://docs.databricks.com/aws/en/sql/language-manual/functions/ai_query#advanced 

In [0]:
%sql

SELECT text, ai_query(
    "databricks-meta-llama-3-3-70b-instruct",
    "주어진 텍스트를 종합적으로 요약하되, 핵심 요점과 주요 아이디어를 간결하게 다루고 관련 세부 사항과 예시를 유지하세요. 불필요한 반복이나 누락 없이 명확하고 정확하도록 하세요." || text,
    modelParameters => named_struct('max_tokens', 100, 'temperature', 0.7)
) AS summary
FROM sudong.apjuice.g_manual;

## Task 기반 AI Function


* 기본 모델: llama, gpt-oss를 기본으로 사용. 변경 될 수 있음
* 한국의 경우 Cross Geography routing이 활성화된 경우 사용 가능
* Compute 비용 + Batch Inference **비용**
* Batch Inferencing사용 시 성능을 위해서 Runtime 15.4 ML LTS를 권고
* ai_parse_document는 llama family 활용, AI agent Bricks enable 필요 (Severless 사용시 별도 조건이 있음))

In [0]:
%sql
select ai_analyze_sentiment("나는 행복해");

In [0]:
%sql
SELECT ai_classify("패스워드가 유출되었다.", ARRAY("urgent", "not urgent"));

In [0]:
%sql
-- entity가 둘 이상인 경우 한개만 
SELECT ai_extract(
    '철수는 수원에서 살고 현대에서 일하고 있습니다. 철수의 email은 chulsu@example.com인데 10분 후에 메일이 올 것입니다.',
    array('사람', '위치', '회사', '이메일', '시간')
  );

In [0]:
%sql
SELECT ai_fix_grammar('나는 내일 ㅂ밟 먹었습니다');

In [0]:
%sql
SELECT ai_gen('너는 Databricks 교육자야, Databricks AI 장점에 대해 말해야해');

In [0]:
%sql
SELECT ai_mask(
    '영희는 서울에 살고 이메일은 yyyy@example.com',
    array('사람', '이메일')
  );

In [0]:
%sql
-- LLM 기반이므로 오류 가능성이 있음

-- 고밀도/저해상도 문서 → 처리 지연 또는 일부 콘텐츠 무시 될 수 있음

-- 지원하지 않는 입력 형식은 불가 (현재 PDF, JPG/JPEG, PNG만 지원)

-- 사용되는 모델에 대한 커스터마이징 불가 및 별도의 커스텀 모델 사용 불가

-- 이미지 처리시 한글, 일본어 인식 성능 제한

-- 디지털 서명 포함 문서는 처리 정확도 떨어질 수 있음
WITH corpus AS (
  SELECT
    path,
    ai_parse_document(content) AS parsed
  FROM
    READ_FILES('/Volumes/sudong/genai/sample_image/apjuice_pdf/', format => 'binaryFile')
)
SELECT
  path,
  parsed:document:pages,
  parsed:document:elements,
  parsed:corrupted_data,
  parsed:error_status,
  parsed:metadata
FROM corpus

In [0]:
%sql
SELECT ai_similarity('안녕', '안녕하세요');

In [0]:
%sql
SELECT ai_summarize(
   '기상청은 전날 밤부터 이날 아침 사이 경기동부와 충청권내륙, 전라권내륙, 경북중·북부내륙, 경남서부내륙을 중심으로 짙은 안개가 끼는 곳이 있겠다고 예보했다.
 
특히 오전까지 낮은 구름대가 유입되는 강원산지에는 가시거리 200m 미만의 짙은 안개가 끼는 곳이 있겠으니 교통안전에 각별히 유의해야겠다.
 
이날 오전까지 강원영동에는 비가 내리겠다.
 
밤부터 오는 22일 아침 사이에는 제주도에도 가끔 비가 내리겠다. 예상 강수량은 5~20㎜다.
 
아침 최저기온은 13~20도, 낮 최고기온은 23~28도 사이를 오르내리겠다.
 
주요 지역 아침 최저기온은 서울 16도, 인천 17도, 수원 16도, 춘천 14도, 강릉 17도, 청주 18도, 대전 17도, 전주 17도, 광주 18도, 대구 18도, 부산 20도, 제주 22도다.
 
낮 최고기온은 서울 26도, 인천 26도, 수원 26도, 춘천 26도, 강릉 24도, 청주 27도, 대전 26도, 전주 27도, 광주 27도, 대구 26도, 부산 27도, 제주 27도다.
 
미세먼지 농도는 전 권역이 좋음 수준으로 전망된다.',
    100
  );

In [0]:
%sql
-- 한국어로 번역은 현재 지원되지 않음
SELECT ai_translate('기상청은 일부 지역에 짙은 안개가 예보되었으며, 강원산지와 제주도에는 비가 내릴 것으로 예상된다. 아침 최저기온은 13~20도, 낮 최고기온은 23~28도 사이를 오르내릴 것으로 예상된다. 미세먼지 농도는 전 권역이 좋음 수준으로 전망된다.', 'en');

In [0]:
%sql

-- sql warehouse에서만 사용 가능
-- Prophet과 유사한 모델 사용
WITH aggregated AS (
  SELECT
    DATE(tpep_pickup_datetime) AS ds,
    dropoff_zip,
    SUM(fare_amount) AS revenue,
    COUNT(*) AS n_trips
  FROM samples.nyctaxi.trips
  GROUP BY 1, 2
)
SELECT * FROM ai_forecast(
  TABLE(aggregated),
  horizon => '2016-03-31',
  time_col => 'ds',
  value_col => ARRAY('revenue', 'n_trips'),
  group_col => 'dropoff_zip',
  prediction_interval_width => 0.9,
  parameters => '{"global_floor": 0}'
) where revenue_forecast is not null;

In [0]:
%sql
-- Querying DIRECT_ACCESS index types are not supported.
-- Input parameters filters_json or columns are not supported.
-- Vector Search with num_results greater than 100 are not supported.
-- provisioned throughput 미지원
SELECT * FROM vector_search(
  index => 'sudong.apjuice.g_manual_index',
  query_text => '착즙 세척 장비 세척 방법',
  query_type => 'HYBRID',
  num_results => 3
);